# Pre-requisites:
Ensure that you have the following pre-requisites to run this notebook:

* Install two open source libraries in your terminal

Run 'pip install yahoo_fin'

Run 'pip install requests_html'

* Connection information to your (local) PostgreSQL database. You will need it for your config.py file that resides in the same folder as this notebook. Example:

API key for yahoofinanceapi web site

API_KEY = "######"

username = "######"

password = "######"

host = "localhost"

port = 5432

database = "######"


# 1. TRENDING STOCK SYMBOLS/TICKERS

API domain: https://yfapi.net
 API path: /v1/finance/trending/{region}

Pull data (extract) from yahoofinancials API (URL path is /v1/finance/trending/{region}).
It returns stock symbols (up to 20) for regions around the world: US AU CA FR DE HK IT ES GB IN.
Pulling for each region.

In [1]:
# Import required modules and variables
import pandas as pd
import requests
import pprint
from config import API_KEY
from config import username, password, host, port, database

In [2]:
# Start with an empty dataframe that will contain the results
# trending_df = pd.DataFrame({'symbol': 'RM', 'region': 'RM', 'startInterval': 12, 'jobTimestamp': 13 }, index=['z'])
financial_trends_df = pd.DataFrame({'symbol': [], 'region': [], 'startInterval': [], 'jobTimestamp': [] })
financial_trends_df


,symbol,region,startInterval,jobTimestamp


In [3]:
# Define connection variables
regions = ["US", "AU", "CA", "FR", "DE", "HK", "IT", "ES", "GB", "IN"]
# regions = ["US", "CA"]

url = "https://yfapi.net/v1/finance/trending/"

# Credentials to include
headers = {
    'x-api-key': API_KEY
    }

In [4]:
# trending_dict = {}
## Make API request for the number of regions of interest. Recursively save them
##  into the dataframe by using 'append'
for region in regions:
    url_string = url + region
    response = requests.request("GET", url_string, headers=headers).json()
#     trending_dict[region] = response['finance']['result']
    response1_df = pd.DataFrame(response['finance']['result'][0]['quotes'])
    response1_df['region'] = region
    response1_df['startInterval'] = response['finance']['result'][0]['startInterval']
    response1_df['jobTimestamp'] = response['finance']['result'][0]['jobTimestamp']
    financial_trends_df = financial_trends_df.append(response1_df, ignore_index=True)

    

In [5]:
## How many responses did we get?  10 regions
len(financial_trends_df['region'].unique())

10

In [ ]:
# TRANSFORMATION: change the date/time on startInterval and jobTimestamp

In [6]:
# # Extraction of the timestamp, startInterval
# from datetime import datetime
# start_interval = response1_df['startInterval'].to_string()
# job_timestamp = response1_df['jobTimestamp']

# job_timestamp = pd.to_datetime(job_timestamp, unit='ms')
# # start_interval = datetime.strptime(start_interval, '%Y%m%d%H%M')
# start_interval, job_timestamp

In [6]:
# Loop through each stock symbol and make a list
stock_list = [stock for stock in financial_trends_df['symbol'].unique()]
stock_list

['BB',
 'FB',
 'HOOD',
 'SOFI',
 'TOST',
 'U',
 'FRSH',
 'ATER',
 'DVAX',
 'PFE',
 'SLI',
 'TLRY',
 'ONTX',
 'WKHS',
 'PTON',
 'CEI',
 'GOTU',
 'NET',
 'FCEL',
 'GOEV',
 '^AORD',
 'BIGG.CN',
 'BB.TO',
 'BTE.TO',
 'NVDA',
 'CL=F',
 'AAPL',
 'LSPD.TO',
 '^FCHI',
 'AMC',
 '3333.HK',
 '0708.HK',
 '6666.HK',
 '0136.HK',
 '0388.HK',
 '0700.HK',
 '3988.HK',
 '1299.HK',
 '0823.HK',
 '0175.HK',
 '0883.HK',
 '0005.HK',
 '0016.HK',
 '9988.HK',
 '1810.HK',
 '^HSI',
 '0939.HK',
 '2800.HK',
 '0857.HK',
 '0303.HK',
 '^IXIC',
 'BABA',
 '^DJI']

In [7]:
# How many unique stock tickers do we have?
len(stock_list)

53

In [8]:
# The quote API query wants the list of symbols as a single comma separated string 
#  with no spaces.  Max is 10 per request.
query_string=''
for stock in stock_list:
    query_string += f"{stock},"
query_string = query_string.rstrip(',')
query_string

'BB,FB,HOOD,SOFI,TOST,U,FRSH,ATER,DVAX,PFE,SLI,TLRY,ONTX,WKHS,PTON,CEI,GOTU,NET,FCEL,GOEV,^AORD,BIGG.CN,BB.TO,BTE.TO,NVDA,CL=F,AAPL,LSPD.TO,^FCHI,AMC,3333.HK,0708.HK,6666.HK,0136.HK,0388.HK,0700.HK,3988.HK,1299.HK,0823.HK,0175.HK,0883.HK,0005.HK,0016.HK,9988.HK,1810.HK,^HSI,0939.HK,2800.HK,0857.HK,0303.HK,^IXIC,BABA,^DJI'

In [9]:
# For now, get the first 10 and run the quote API
query_string=''
for i in range(10):
    query_string += f"{stock_list[i]},"
query_string = query_string.rstrip(',')
query_string

'BB,FB,HOOD,SOFI,TOST,U,FRSH,ATER,DVAX,PFE'

In [29]:
financial_trends_df.head()

,symbol,region,startInterval,jobTimestamp
0,BB,US,2.021092e+11,1.632367e+12
1,FB,US,2.021092e+11,1.632367e+12
2,HOOD,US,2.021092e+11,1.632367e+12
3,SOFI,US,2.021092e+11,1.632367e+12
4,TOST,US,2.021092e+11,1.632367e+12


# 2. REAL TIME QUOTE DATA

API domain: https://yfapi.net
API path: /v6/finance/quote
API variables:  querystring  {"symbols" : "string1,string2,string3,..."}

Pull data (extract) from yahoofinancials API (URL path is /v6/finance/quote) with "symbols" set to comma-separated list of up to 10 stock symbols at a time.

Use the query_string of 10 stock symbols.

In [10]:
## Query of /v6/finance/quote API

url = "https://yfapi.net/v6/finance/quote"

## Warning: no space between symbols
querystring = {"symbols" : query_string}

# Credentials to include
headers = {
    'x-api-key': API_KEY
    }

In [11]:
## Make API request (query string required for this API)
response2 = requests.request("GET", url, headers=headers, params=querystring).json()

# What does the response look like?  We will have to use indexing to get to the 'result' level
pprint.pprint(response2)

{'quoteResponse': {'error': None,
                   'result': [{'ask': 10.16,
                               'askSize': 40,
                               'averageAnalystRating': '3.7 - Underperform',
                               'averageDailyVolume10Day': 8098657,
                               'averageDailyVolume3Month': 10418568,
                               'bid': 10.15,
                               'bidSize': 10,
                               'bookValue': 2.571,
                               'currency': 'USD',
                               'displayName': 'BlackBerry',
                               'earningsTimestamp': 1632330304,
                               'earningsTimestampEnd': 1640552400,
                               'earningsTimestampStart': 1640120400,
                               'epsCurrentYear': -0.14,
                               'epsForward': 0.1,
                               'epsTrailingTwelveMonths': -0.944,
                               'esgPop

In [12]:
## How many records did we get?
len(response2['quoteResponse']['result'])

10

In [13]:
# Create a new DataFrame from all the results in the dictionary
response2_df = pd.DataFrame(response2['quoteResponse']['result'])
response2_df

,language,region,quoteType,quoteSourceName,triggerable,currency,forwardPE,priceToBook,sourceInterval,exchangeDataDelayedBy,...,displayName,symbol,trailingPE,ipoExpectedDate,prevName,nameChangeDate,newListingDate,dividendDate,trailingAnnualDividendRate,trailingAnnualDividendYield
0,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,95.600006,3.718398,15,0,...,BlackBerry,BB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,21.330639,7.016744,15,0,...,Facebook,FB,25.487152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,en-US,US,EQUITY,Delayed Quote,True,USD,-520.888850,-5.493965,15,0,...,Robinhood Markets,HOOD,NaN,2021-07-29,NaN,NaN,NaN,NaN,NaN,NaN
3,en-US,US,EQUITY,Delayed Quote,True,USD,-240.571430,3.180960,15,0,...,SoFi,SOFI,NaN,2021-06-01,Social Capital Hedosophia Holdings Corp. V,2021-09-22,NaN,NaN,NaN,NaN
4,en-US,US,EQUITY,Delayed Quote,True,USD,NaN,-4.543208,15,0,...,Toast,TOST,NaN,2021-09-22,NaN,NaN,2021-09-22,NaN,NaN,NaN
5,en-US,US,EQUITY,Delayed Quote,True,USD,-1027.000000,19.124767,15,0,...,Unity Software,U,NaN,2020-09-18,NaN,NaN,NaN,NaN,NaN,NaN
6,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,NaN,-1.197009,15,0,...,NaN,FRSH,NaN,2021-09-22,NaN,NaN,2021-09-22,NaN,NaN,NaN
7,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,-8.156097,3.072400,15,0,...,Aterian,ATER,NaN,NaN,"Mohawk Group Holdings, Inc.",2021-09-22,NaN,NaN,NaN,NaN
8,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,12.782313,25.845943,15,0,...,Dynavax,DVAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,en-US,US,EQUITY,Delayed Quote,True,USD,12.208334,3.513190,15,0,...,Pfizer,PFE,18.766012,NaN,NaN,NaN,NaN,1.630973e+09,1.54,0.035064


In [14]:
# Get the finance_quote_df
finance_quote_df = response2_df.copy()
finance_quote_df.head()

,language,region,quoteType,quoteSourceName,triggerable,currency,forwardPE,priceToBook,sourceInterval,exchangeDataDelayedBy,...,displayName,symbol,trailingPE,ipoExpectedDate,prevName,nameChangeDate,newListingDate,dividendDate,trailingAnnualDividendRate,trailingAnnualDividendYield
0,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,95.600006,3.718398,15,0,...,BlackBerry,BB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en-US,US,EQUITY,Nasdaq Real Time Price,True,USD,21.330639,7.016744,15,0,...,Facebook,FB,25.487152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,en-US,US,EQUITY,Delayed Quote,True,USD,-520.888850,-5.493965,15,0,...,Robinhood Markets,HOOD,NaN,2021-07-29,NaN,NaN,NaN,NaN,NaN,NaN
3,en-US,US,EQUITY,Delayed Quote,True,USD,-240.571430,3.180960,15,0,...,SoFi,SOFI,NaN,2021-06-01,Social Capital Hedosophia Holdings Corp. V,2021-09-22,NaN,NaN,NaN,NaN
4,en-US,US,EQUITY,Delayed Quote,True,USD,NaN,-4.543208,15,0,...,Toast,TOST,NaN,2021-09-22,NaN,NaN,2021-09-22,NaN,NaN,NaN


In [16]:
finance_quote_df.columns

Index(['language', 'region', 'quoteType', 'quoteSourceName', 'triggerable',
       'currency', 'forwardPE', 'priceToBook', 'sourceInterval',
       'exchangeDataDelayedBy', 'averageAnalystRating', 'tradeable',
       'shortName', 'postMarketChangePercent', 'postMarketTime',
       'postMarketPrice', 'postMarketChange', 'regularMarketChange',
       'regularMarketChangePercent', 'regularMarketTime', 'regularMarketPrice',
       'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow',
       'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask',
       'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency',
       'regularMarketOpen', 'averageDailyVolume3Month',
       'averageDailyVolume10Day', 'fiftyTwoWeekLowChange',
       'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange',
       'fiftyTwoWeekHighChange', 'fiftyTwoWeekHighChangePercent',
       'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'earningsTimestamp',
       'earningsTimestampStart', 'earningsTim

# 3. STOCK HISTORY

Open Source Library:  yahoo_fin
Usage: get_data(ticker, start_date = mm/dd/YYYY, end_date = None, index_as_date = True, interval = “1d”)
API variables:  ticker      a single ticker symbol or a list of ticker symbols
                start_date  first date in requested history, back to 1970
                end_date    last date in requested hostory, to the present
                index_as_date  should date column be used as the dataframe index?
                interval    data interval of 1d, 1w, 1m for daily, weekly, monthly

Pull data (extract) from yahoo_fin library with "ticker" set to a list of up to 10 stock symbols at a time.  Specify data interval and range of desired time span.

Returns one dataframe of data per ticker

In [17]:
## Query of yahoo_fin
from yahoo_fin.stock_info import get_data

my_tickers=stock_list
my_startdate="12/04/2009"
my_enddate="09/21/2021"
my_indexdate=True
my_interval="1wk"

In [18]:
## Make API request

response3 = {}
for ticker in my_tickers:
    try:
        response3[ticker] = get_data(ticker, 
                                     start_date=my_startdate,
                                     end_date=my_enddate,
                                     index_as_date=my_indexdate,
                                     interval=my_interval)
        print(f"Good response on ticker {ticker}")
    except:
        print(f"No data on ticker {ticker}...")

Good response on ticker BB
Good response on ticker FB
Good response on ticker HOOD
Good response on ticker SOFI
No data on ticker TOST...
Good response on ticker U
No data on ticker FRSH...
Good response on ticker ATER
Good response on ticker DVAX
Good response on ticker PFE
Good response on ticker SLI
Good response on ticker TLRY
Good response on ticker ONTX
Good response on ticker WKHS
Good response on ticker PTON
Good response on ticker CEI
Good response on ticker GOTU
Good response on ticker NET
Good response on ticker FCEL
Good response on ticker GOEV
Good response on ticker ^AORD
Good response on ticker BIGG.CN
Good response on ticker BB.TO
Good response on ticker BTE.TO
Good response on ticker NVDA
Good response on ticker CL=F
Good response on ticker AAPL
Good response on ticker LSPD.TO
Good response on ticker ^FCHI
Good response on ticker AMC
Good response on ticker 3333.HK
Good response on ticker 0708.HK
Good response on ticker 6666.HK
Good response on ticker 0136.HK
Good resp

In [19]:
## How many records did we get?
len(response3)

51

In [20]:
# response3 is a dictionary of dataframes.  Using the .keys you can see which
#  tickers actually returned some data
response3.keys()

dict_keys(['BB', 'FB', 'HOOD', 'SOFI', 'U', 'ATER', 'DVAX', 'PFE', 'SLI', 'TLRY', 'ONTX', 'WKHS', 'PTON', 'CEI', 'GOTU', 'NET', 'FCEL', 'GOEV', '^AORD', 'BIGG.CN', 'BB.TO', 'BTE.TO', 'NVDA', 'CL=F', 'AAPL', 'LSPD.TO', '^FCHI', 'AMC', '3333.HK', '0708.HK', '6666.HK', '0136.HK', '0388.HK', '0700.HK', '3988.HK', '1299.HK', '0823.HK', '0175.HK', '0883.HK', '0005.HK', '0016.HK', '9988.HK', '1810.HK', '^HSI', '0939.HK', '2800.HK', '0857.HK', '0303.HK', '^IXIC', 'BABA', '^DJI'])

In [21]:
# Concatenate all dataframes
# Start with an empty dataframe that will contain the results
stock_info_df = pd.DataFrame({'open': [], 'high': [], 'low': [], 'close': [], 'adjclose': [], 'volume': [], 'ticker': [] })

# Loop through the stock tickers that received a response and add them to
#   stock_info_df
for stock in response3.keys():
    stock_info_df = stock_info_df.append(response3[stock])
len(stock_info_df)

23177

In [25]:
stock_info_df.head()

,open,high,low,close,adjclose,volume,ticker
2009-11-30,59.799999,60.290001,58.130001,58.750000,58.750000,27307600.0,BB
2009-12-07,59.040001,66.500000,58.860001,63.840000,63.840000,99963600.0,BB
2009-12-14,64.059998,71.599998,62.529999,70.000000,70.000000,147390400.0,BB
2009-12-21,70.330002,71.540001,66.500000,66.919998,66.919998,62976600.0,BB
2009-12-28,67.410004,68.690002,66.500000,67.540001,67.540001,34025200.0,BB


# 4. Steps beyond

We could save the raw outputs into 3 database tables.

Transformations (either in the database or here in Jupyter Notebook):
The above new API pulls in dataframes with the proper timestamp format.  With this API we don't need to convert the time stamp

Transforming: renaming columns, taking fewer columns into new dataframe before loading, calculating new needed columns, dropping duplicates

Load (in the database):
Decide for each raw table above which columns will be needed for the final set of tables.  Make those tables, calculations, etc, and produce the final set of tables.  Set up connection string and method for how to transfer dataframe into database tables.

# 5. Connect to database
Use the values pulled in from config.py to construct the database connection string.  Connect to the database.

In [22]:
from sqlalchemy import create_engine

# Define the engine string to be used for connecting to your database using the 
#  environment variables found in config.py

# database_url = f'postgresql://{username}:{password}@{host}:{port}/{database}'
# engine = create_engine(database_url)
# connection = engine.connect()
connection_string = f'{username}:{password}@{host}:{port}/{database}'
engine = create_engine(f'postgresql://{connection_string}')

In [23]:
# Write dataframes to database, decide whether index is true or false, depending on
#   whether the index should be a column or not (possible primary key column)
financial_trends_df.to_sql(name='financial_trends', con=engine, if_exists='append', index=False)

In [24]:
finance_quote_df.to_sql(name='finance_quote', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "quoteType" of relation "finance_quote" does not exist
LINE 1: INSERT INTO finance_quote (language, region, "quoteType", "q...
                                                     ^

[SQL: INSERT INTO finance_quote (language, region, "quoteType", "quoteSourceName", triggerable, currency, "forwardPE", "priceToBook", "sourceInterval", "exchangeDataDelayedBy", "averageAnalystRating", tradeable, "shortName", "postMarketChangePercent", "postMarketTime", "postMarketPrice", "postMarketChange", "regularMarketChange", "regularMarketChangePercent", "regularMarketTime", "regularMarketPrice", "regularMarketDayHigh", "regularMarketDayRange", "regularMarketDayLow", "regularMarketVolume", "regularMarketPreviousClose", bid, ask, "bidSize", "askSize", "fullExchangeName", "financialCurrency", "regularMarketOpen", "averageDailyVolume3Month", "averageDailyVolume10Day", "fiftyTwoWeekLowChange", "fiftyTwoWeekLowChangePercent", "fiftyTwoWeekRange", "

In [25]:
stock_info_df.to_sql(name='stock_info', con=engine, if_exists='append', index=False)

In [26]:
# Verify that there is data in the SQL tables
pd.read_sql_query('select * from financial_trends', con=engine).head()

,symbol,region,startInterval,jobTimestamp
0,BB,US,2.021092e+11,1.632363e+12
1,FB,US,2.021092e+11,1.632363e+12
2,HOOD,US,2.021092e+11,1.632363e+12
3,TOST,US,2.021092e+11,1.632363e+12
4,SOFI,US,2.021092e+11,1.632363e+12


In [27]:
# Verify that there is data in the SQL tables
pd.read_sql_query('select * from finance_quote', con=engine).head()

,ask,asksize,averageanalystrating,averagedailyvolume10day,averagedailyvolume3month,bid,bidsize,bookvalue,currency,displayname,...,symbol,tradeable,trailingannualdividendrate,trailingannualdividendyield,trailingpe,triggerable,twohundreddayaverage,twohundreddayaveragechange,twohundreddayaveragechangepercent,jobtimestamp


In [28]:
# Verify that there is data in the SQL tables
pd.read_sql_query('select * from stock_info', con=engine).head()

,open,high,low,close,adjclose,volume,ticker
0,42.049999,45.000000,38.000000,38.230000,38.230000,573576400.0,FB
1,36.529999,36.660000,30.940001,31.910000,31.910000,430966300.0,FB
2,31.480000,31.690001,26.830000,27.719999,27.719999,288826000.0,FB
3,27.200001,27.760000,25.520000,27.100000,27.100000,203386400.0,FB
4,27.180000,30.100000,26.840000,30.010000,30.010000,121558000.0,FB
